# **Notebook 3: Machine Learning Model**

## Objectives

* Create ML model for predicting future CO2 levels based on current data

## Inputs

* clean_data.csv produced by Notebook 1

## Outputs

* Generates a predictive ML model for use with climate data 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Import Packages and Data

Import packages needed to run this notebook

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


Import cleaned dataset

In [ ]:
#set path to clean data file
path = "../raw_data/clean_data.csv"

# read data into dataframe
df_clean = pd.read_csv(path)

# display data types to see if any need to be changed
df_clean.info()

Confirm the new current directory

# Section 1

Section 1 content

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                    df.drop(['target'], axis=1),
                                    df['target'],
                                    test_size=0.2,
                                    random_state=101
                                    )

print(
    "* Train set:",
    X_train.shape,
    y_train.shape,
    "\n* Test set:",
    X_test.shape,
    y_test.shape,
)

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
